In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as pyplot
import os
import re
import math
import time
from enum import Enum

# Pomožne funkcije

Globalne spremenljivke za direktorij z vhodnimi slikami in velikost filtra.

V direktorij, ki je definiran z `ASSETS_FOLDER` se za določena področja pridobijo izračunani indeksi, s pomočjo prve naloge.

Direktorij mora obstajati za pravilno delovanje programa.

Aplikacija podpira filtre, kjer se s pomočjo vhodnega parametra S izračuna velikost po formuli:

$ SIZE = 2 * S + 1 $

$ SHAPE = (SIZE, SIZE) $

In [ ]:
ASSETS_FOLDER = "assets/"
S = 3

SHAPE = (2 * S + 1, 2 * S + 1)

Enum razred, ki vsebuje vse podprte filtre.

In [ ]:
class Filter(Enum):
  RECTANGLE = cv2.MORPH_RECT
  CROSS = cv2.MORPH_CROSS
  ELLIPSE = cv2.MORPH_ELLIPSE

Pomožna funkcija za prikaz slike.

In [ ]:
def display_image(image, title):
  imgplot = pyplot.imshow(image, cmap="gray")
  pyplot.axis('off')
  pyplot.title(title)
  pyplot.show()

Slike so normalizirane na interval `[0, 1]`, vendar lahko pri normalizaciji pride do napak, zato se vse vrednosti, ki so izven tega intervala popravijo na ustrezne vrednosti.

In [ ]:
def fix_image(image):
  image[image > 1] = 1
  image[image <= 0] = 0
  return image

Funkcija, ki naloži sliko iz diska in jo normalizira.

In [ ]:
def load_image(filename):
  image = cv2.imread(ASSETS_FOLDER + filename, -1)

  if image is not None:
    return image

Funkcija, ki dobi kot parametra dve sliki enake velikosti, kot rezultat pa vrne sliko, ki za vsak piksel vsebuje največjo vrednost izmed dveh vhodnih parametrov.

In [ ]:
def get_max(first, second):
  return np.maximum(first, second)

Funkcija, ki dobi kot parametra dve sliki enake velikosti, kot rezultat pa vrne sliko, ki za vsak piksel vsebuje najmanjšo vrednost izmed dveh vhodnih parametrov.

In [ ]:
def get_min(first, second):
  return np.minimum(first, second)

Aplikacija omogoča uporabo večih različnih tipov filtrov, natančneje tistih, ki so definirani v zgornjem Enumu, tukaj pa se s pomočjo knjižnice OpenCV generirajo glede na podano velikost in podan tip filtra.

In [ ]:
def get_filter(shape, type):
  return cv2.getStructuringElement(type, shape)

Pomožna funkcija za krčenje nad podano sliko s podanim filtrom.

Krčenje za vsak piksel v sliki pregleda okolico, ki je definirana z enicami v filtru in kot izhod vrne najmanjši piksel.

In [ ]:
def erode(image, filter):
  return cv2.erode(image, filter, iterations=1)

Pomožna funkcija za širjenje nad podano sliko s podanim filtrom.

Širjenje za vsak piksel v sliki pregleda okolico, ki je definirana z enicami v filtru in kot izhod vrne največji piksel.

In [ ]:
def dilate(image, filter):
  return cv2.dilate(image, filter, iterations=1)

Pomožna funkcija za odpiranje nad podano sliko s podanim filtrom prvo izvede krčenje, nato pa širjenje.

In [ ]:
def opening(image, filter):
  eroded = erode(image, filter)
  return dilate(eroded, filter)

Pomožna funkcija za odpiranje nad podano sliko s podanim filtrom prvo izvede širjenje, nato pa krčenje.

In [ ]:
def closing(image, filter):
  dilated = dilate(image, filter)
  return erode(dilated, filter)

Sledi odpiranje z rekonstrukcijo, ki deluje po naslednjem postopku:
1. Nad vhodno sliko izvede krčenje s podanim filtrom
2. Nad rezultatom iz 1. koraka izvede širjenje s poljubnim filtrom velikosti `S = 1`, kar po enačbi zapisani zgoraj vrne filter oblike `(3, 3)` 
3. Z vhodno sliko in rezultatom iz 2. koraka v novo sliko zapiše najmanjše vrednosti enakoležečih pikslov
4. Drugi in tretji korak se izvajata dokler nista dva zaporedna izhoda enaka

In [ ]:
def opening_with_reconstruction(image, filter, f):
  start = time.time()
  reconstructed = erode(image, filter)
  previous = None

  iterations = 0
  equal = 0

  while equal < 2:
    iterations += 1

    previous = reconstructed
    max = dilate(reconstructed, get_filter((3, 3), f.value))
    reconstructed = get_min(image, max)

    if np.array_equal(previous, reconstructed):
      equal += 1
    else: 
      equal = 0

  end = time.time()
  duration = end - start

  print("OPENING WITH RECONSTRUCTION WITH " + f.name + " FILTER TOOK " + str(iterations) + " ITERATIONS AND TOOK " + str(duration) + " SECONDS")

  return reconstructed

In še zapiranje z rekonstrukcijo, ki deluje po naslednjem postopku:
1. Nad vhodno sliko izvede širjenje s podanim filtrom
2. Nad rezultatom iz 1. koraka izvede krčenje s poljubnim filtrom velikosti `S = 1`, kar po enačbi zapisani zgoraj vrne filter oblike `(3, 3)` 
3. Z vhodno sliko in rezultatom iz 2. koraka v novo sliko zapiše največje vrednosti enakoležečih pikslov
4. Drugi in tretji korak se izvajata dokler nista dva zaporedna izhoda enaka

In [ ]:
def closing_with_reconstruction(image, filter, f):
  start = time.time()
  reconstructed = dilate(image, filter)
  previous = None
  
  iterations = 0
  equal = 0

  while equal < 2:
    iterations += 1

    previous = reconstructed
    min = erode(reconstructed, get_filter((3, 3), f.value))
    reconstructed = get_max(image, min)

    if np.array_equal(previous, reconstructed):
      equal += 1
    else: 
      equal = 0

  end = time.time()
  duration = end - start

  print("CLOSING WITH RECONSTRUCTION WITH " + f.name + " FILTER TOOK " + str(iterations) + " ITERATIONS AND TOOK " + str(duration) + " SECONDS")

  return reconstructed

# Glavni del programa

V tem kratkem odseku pa se za vse slike v direktorju `ASSETS_FOLDER` izvedejo operacije krčenja, širjenja, odpiranja, zapiranja, odpiranja z rekonstrukcijo in zapiranja z rekonstrukcijo.

In [ ]:
for filename in os.listdir(ASSETS_FOLDER):
  image = load_image(filename)

  if image is not None:
    name = filename.split(".")
    print("------------------------------------")
    print("HANDLING: " + filename)
    for filter in Filter:
      print("HANDLING FILTER: " + filter.name)
      
      kernel = get_filter(SHAPE, filter.value)
      
      display_image(image, "ORIGINAL (" + name[0] + ")")

      handled = erode(image, kernel)
      display_image(handled, "ERODED WITH " + filter.name + " FILTER")

      handled = dilate(image, kernel)
      display_image(handled, "DILATED WITH " + filter.name + " FILTER")

      handled = opening(image, kernel)
      display_image(handled, "OPENING WITH " + filter.name + " FILTER")

      handled = closing(image, kernel)
      display_image(handled, "CLOSING WITH " + filter.name + " FILTER")

      handled = opening_with_reconstruction(image, kernel, filter)
      display_image(handled, "OPENING WITH RECONSTRUCTION WITH " + filter.name + " FILTER")

      handled = closing_with_reconstruction(image, kernel, filter)
      display_image(handled, "CLOSING WITH RECONSTRUCTION WITH " + filter.name + " FILTER")